In [48]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [3]:
krajDf = pd.read_excel("data/kraj_okres_link.xlsx")


,Kraj,Okres,Link,Maxpage
0,Bratislavsky Kraj,Bratislava 1,https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,7
1,Bratislavsky Kraj,Bratislava 2,https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,8
2,Bratislavsky Kraj,Bratislava 3,https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,7
3,Bratislavsky Kraj,Bratislava 4,https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,3
4,Bratislavsky Kraj,Bratislava 5,https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,5
...,...,...,...,...
75,Žilinský,Námestovo,https://www.e-vuc.sk/zsk/zdravotnictvo/ambulan...,1
76,Žilinský,Ružomberok,https://www.e-vuc.sk/zsk/zdravotnictvo/ambulan...,3
77,Žilinský,Turčianske Teplice,https://www.e-vuc.sk/zsk/zdravotnictvo/ambulan...,1
78,Žilinský,Tvrdošín,https://www.e-vuc.sk/zsk/zdravotnictvo/ambulan...,1


In [50]:
%%time
for index, krajRow in krajDf.iterrows():
    kraj = krajRow['Kraj']
    okres = krajRow['Okres']
    row_link = krajRow['Link']
    max_page = krajRow['Maxpage']
    
    print(index,"80",kraj, okres)
    data = []

    for page in range(1,max_page+1):
        url = f"{row_link}&page={page}"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")

        
        table = soup.find('table', attrs={'class':'list'})

        rows = table.find_all('tr')
        for row in rows:
            title_html = row.find('a')
            
            title = title_html.text.strip()
            
            doctor_url = ""
            if title_html.has_attr('href'):
                doctor_url = title_html['href']
            
            try:
                specialization = row.find('span').text.replace("(", " ").replace(")", "")
            except:
                specialization = ""

            data_row = {
                "title" : title,
                "doctor_url" : doctor_url,
                "specialization" : specialization,
                "kraj" : kraj,
                "okres" : okres,

            }
            data.append(data_row)
        time.sleep(2)

    df = pd.DataFrame(data)
    df.to_csv(f"data/kraj_okres_data/{kraj}_{okres}_doctor_list.csv",index=False)



0 80 Bratislavsky Kraj Bratislava 1
1 80 Bratislavsky Kraj Bratislava 2
2 80 Bratislavsky Kraj Bratislava 3
3 80 Bratislavsky Kraj Bratislava 4
4 80 Bratislavsky Kraj Bratislava 5
5 80 Bratislavsky Kraj Malacky
6 80 Bratislavsky Kraj Pezinok
7 80 Bratislavsky Kraj Senec
8 80 Banskobystricky Banská Bystrica
9 80 Banskobystricky Banska Štiavnica
10 80 Banskobystricky Brezno
11 80 Banskobystricky Detva
12 80 Banskobystricky Krupina
13 80 Banskobystricky Lučenec
14 80 Banskobystricky Poltár
15 80 Banskobystricky Revúca
16 80 Banskobystricky Rimavska Sobota
17 80 Banskobystricky Veľký Krtíš
18 80 Banskobystricky Žarnovica
19 80 Banskobystricky Žiar nad Hronom
20 80 Banskobystricky Zvolen
21 80 Košický Gelnica
22 80 Košický Košice - okolie
23 80 Košický Košice 1
24 80 Košický Košice 2
25 80 Košický Košice 3
26 80 Košický Košice 4
27 80 Košický Michalovce
28 80 Košický Rožňava
29 80 Košický Sobrance
30 80 Košický Spišská Nová Ves
31 80 Košický Trebišov
32 80 Nitriansky Komárno
33 80 Nitriansk

In [100]:
import glob
import os

files = os.path.join("data/kraj_okres_data/", "*.csv")

# list of merged files returned
files = glob.glob(files)

df_all = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)
df_all

,title,doctor_url,specialization,kraj,okres
0,"Gynekologická ambulancia, Bratislava-Staré Mes...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Gynekológia a pôrodníctvo,Bratislavsky Kraj,Bratislava 1
1,"Urologická ambulancia, Bratislava-Staré Mesto,...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Urológia,Bratislavsky Kraj,Bratislava 1
2,"Stomatologická ambulancia, Bratislava-Staré Me...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Zubné lekárstvo,Bratislavsky Kraj,Bratislava 1
3,"SVaLZ, fyziatria, balneológia a liečebná rehab...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,"Fyziatria, balneológia a liečebná rehabilitácia",Bratislavsky Kraj,Bratislava 1
4,"Otorinolaryngologická ambulancia, Bratislava-S...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Otorinolaryngológia,Bratislavsky Kraj,Bratislava 1
...,...,...,...,...,...
22768,"Oftalmologická ambulancia, MUDr. Anna Fekésház...",https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Oftalmológia,Prešovský,Humenné
22769,"ADOS, Nemocničná, Humenné, (ADOS-POMOC s.r.o.)",https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Domáca ošetrovateľská starostlivosť,Prešovský,Humenné
22770,Ambulancia pediatrickej pneumológie a ftizeoló...,https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Pediatrická pneumológia a ftizeológia,Prešovský,Humenné
22771,"Všeobecná ambulancia pre deti a dorast, Humenn...",https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Všeobecná starostlivosť o deti a dorast,Prešovský,Humenné


In [69]:
# cat_df = list()
# for x in df_all.specialization.unique():
#     try:
#         cats = x.split(",")
#         for c in cats:
#             if not c.strip().upper() in cat_df:
#                 cat_df.append(c.strip().upper())
#     except:
#         print(x)
# pd.DataFrame(cat_df).to_excel("categories.xlsx",index=False)

nan


287

In [101]:
def isPediater(spec, categories):
    try:
        cats = spec.split(",")
        for c in cats:
            if c.strip().upper() in categories:
                return 1
            else:
                return 0
    except:
        return 0

df_categories = pd.read_excel("categories.xlsx")

df_all['pediater'] = df_all['specialization'].apply(lambda row: isPediater(row,df_categories.values))

In [102]:
from sklearn import preprocessing
le_kraj = preprocessing.LabelEncoder()
le_kraj.fit(df_all['kraj'])
df_all['kraj_id'] = le_kraj.transform(df_all['kraj'])

le_okres = preprocessing.LabelEncoder()
le_okres.fit(df_all['okres'])
df_all['okres_id'] = le_okres.transform(df_all['okres'])


In [103]:

import json
okres_id = [ {"id" : index, "name" : c}for index, c in enumerate(list(le_okres.classes_))]

with open('okres_ids.json', 'w', encoding='utf-8') as f:
    json.dump(okres_id, f, ensure_ascii=False, indent=4)


kraj_id = [ {"id" : index, "name" : c}for index, c in enumerate(list(le_kraj.classes_))]

with open('kraj_ids.json', 'w', encoding='utf-8') as f:
    json.dump(kraj_id, f, ensure_ascii=False, indent=4)



In [105]:
df_all.to_csv("data/all_doctors.csv",index=False)

In [106]:
pediaters_df = df_all[df_all['pediater'] == 1]

In [110]:
pediaters_df.to_csv("data/pediater_doctors.csv",index=False)